# Charmed Kubeflow Databases Migration Guide

There are two databases in Charmed Kubeflow - `katib-db` and `kfp-db`. In Charmed Kubeflow 1.7 these MariaDB databases deployed by `charmed-osm-mariadb-k8s` charm. Since release 1.7 there is an option to relate charm to new MySQL database that is deployed by `mysql-k8s` charm. In cases where previous deployment has MariaDB a migration of database is required.

## Prerequisites

In order to perform migration the following tools should be installed on client machine that has access to Charmed Kubeflow deployment:
- `mysql-client` (install by running `sudo apt install mysql-client`)

## Katib DB migration

In [ ]:
# obtain username and password of existing MariadDB database from DB relation
# in Charmed Kubeflow 1.6 and 1.7 username used is `root` and password is specified in `mysql` relation by
# 'root_password'
KATIB_DB_MYSQL_RELATION_ID=$(juju show-unit katib-db/0 | yq '.[] | .relation-info | select(.[].endpoint == "mysql") | .[0] | .relation-id')
KATIB_DB_USER=root
KATIB_DB_PASSWORD=$(bash -c "juju run --unit katib-db/0 'relation-get -r $KATIB_DB_MYSQL_RELATION_ID - katib-db/0' | grep root_password" | awk '{print $2}')
KATIB_DB_IP=$(juju show-unit katib-db/0 | yq '.[] | .address')

# ensure that there are no new connections are made and that dababase is not altered
# remove relation between katib-db-manager charm and MariaDB charm
juju remove-relation katib-db-manager katib-db

# connect to unit's IP address using mysql client and verify that all data is intact
mysql --host=$KATIB_DB_IP --user=$KATIB_DB_USER --password=$KATIB_DB_PASSWORD

# create backup of all databases file using `mysqldump` utility, username, password, and unit's IP address
# onbtained earlier
mysqldump --host=$KATIB_DB_IP --user=$KATIB_DB_USER --password=$KATIB_DB_PASSWORD --column-statistics=0 --databases katib  > katib-db.sql

# deploy new MySQL database charm
juju deploy mysql-k8s katib-db-mysql --channel 8.0/stable --trust

# obtain username and password of new MySQL database from DB relation
# note that since relation is not yet established it is impossible to retrieve this information from that relation
# the relation can only be established after data is retored into new database
MYSQL_DB_USER=$(juju run-action katib-db-mysql/0 get-password --wait | yq '.[] | .results.username')
MYSQL_DB_PASSWORD=$(juju run-action katib-db-mysql/0 get-password --wait | yq '.[] | .results.password')
MYSQL_DB_IP=$(juju show-unit katib-db/0 | yq '.[] | .address')

# connect to new DB using username, password, and unit's IP address and restore database from backup
mysql --host=$MYSQL_DB_IP --user=$MYSQL_DB_USER --password=$MYSQL_DB_PASSWORD < katib-db.sql

# relate katib-db-manager and new MySQL database charm
juju relate katib-db-manager:relational-db katib-db-mysql:database

## KFP DB migration

In [ ]:
# obtain username and password of existing MariadDB database from DB relation
# in Charmed Kubeflow 1.6 and 1.7 username used is `root` and password is specified in `mysql` relation by
# 'root_password'
KFP_DB_MYSQL_RELATION_ID=$(juju show-unit kfp-db/0 | yq '.[] | .relation-info | select(.[].endpoint == "mysql") | .[0] | .relation-id')
KFP_DB_USER=root
KFP_DB_PASSWORD=$(bash -c "juju run --unit kfp-db/0 'relation-get -r $KFP_DB_MYSQL_RELATION_ID - kfp-db/0' | grep root_password" | awk '{print $2}')
KFP_DB_IP=$(juju show-unit kfp-db/0 | yq '.[] | .address')

# ensure that there are no new connections are made and that dababase is not altered
# remove relation between kfp-api charm and MariaDB charm
juju remove-relation kfp-api kfp-db

# connect to unit's IP address using mysql client and verify that all data is intact
mysql --host=$KFP_DB_IP --user=$KFP_DB_USER --password=$KFP_DB_PASSWORD

# create backup of all databases file using `mysqldump` utility, username, password, and unit's IP address
# onbtained earlier
mysqldump --host=$KFP_DB_IP --user=$KFP_DB_USER --password=$KFP_DB_PASSWORD --column-statistics=0 --databases mlpipeline  > kfp-db.sql

# deploy new MySQL database charm
juju deploy mysql-k8s kfp-db-mysql --channel 8.0/stable --trust

# obtain username and password of new MySQL database from DB relation
# note that since relation is not yet established it is impossible to retrieve this information from that relation
# the relation can only be established after data is retored into new database
MYSQL_DB_USER=$(juju run-action kfp-db-mysql/0 get-password --wait | yq '.[] | .results.username')
MYSQL_DB_PASSWORD=$(juju run-action kfp-db-mysql/0 get-password --wait | yq '.[] | .results.password')
MYSQL_DB_IP=$(juju show-unit kfp-db-mysql/0 | yq '.[] | .address')

# connect to new DB using username, password, and unit's IP address and restore database from backup
mysql --host=$MYSQL_DB_IP --user=$MYSQL_DB_USER --password=$MYSQL_DB_PASSWORD < kfp-db.sql

# relate kfp-api and new MySQL database charm
juju relate kfp-api:relational-db kfp-db-mysql:database